<a href="https://colab.research.google.com/github/dhanuvagman006/Nexa/blob/main/nexav2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
api_key = 'add api key here'

In [ ]:
import json, requests
from openai import OpenAI

# You can use any model that supports tool calling
MODEL = "google/gemini-2.0-flash-001"

openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

In [ ]:
events=["google developers meet 2025 on 26/05/2025"]

In [ ]:
def get_upcoming_events():
  return events


def add_upcomming_events(event_to_add):
    events.append(event_to_add)
    return "Event added successfully"

def find_sub_domains(domain):
  url = "https://subdomain-finder5.p.rapidapi.com/subdomain-finder"

  querystring = {"domain":domain}

  headers = {
    "x-rapidapi-key": "0f74f2e409msh0b29a2c79506d6bp1383b8jsn595e259130b4",
    "x-rapidapi-host": "subdomain-finder5.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  if response.status_code == 200:
      data = response.json()
      subdomains = [entry['subdomain'] for entry in data.get('data', []) if 'subdomain' in entry]
      return(subdomains)
  else:
      return(f"Request failed with status code {response.status_code}")

In [ ]:
get_upcoming_events()

['google developers meet 2025 on 26/05/2025']

In [ ]:
tools = [
    {
        "type":"function",
        "function":{
            "name":"get_upcoming_events",
            "description":"Returns the upcomming events.",
            "parameters":{
                "type":"object",
                "properties":{
                },
                "required":[]
            }
        }
    }
]


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "List all the upcomming events"}
]

# or use input():
# messages=[
#     {"role":"system","content":"You are a helpful robot"},
#     {"role":"user","content":input("Enter your prompt here:")}
# ]



In [ ]:
response = openai_client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"  # LLM picks the right tool
)

tool_call = response.choices[0].message.tool_calls[0]
tool_name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)


if tool_name =="get_upcoming_events":
    tool_result=str(get_upcoming_events())
print(tool_name)

follow_up_messages = messages + [
    response.choices[0].message,  # Tool call message
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_name,
        "content": tool_result
    }
]

final_response = openai_client.chat.completions.create(
    model=MODEL,
    messages=follow_up_messages
)

print(final_response.choices[0].message.content)



get_upcoming_events
OK. I see one upcoming event: google developers meet 2025 on 26/05/2025.



In [ ]:
find_sub_domains("swizosoft.com")

{'info': {'scan_started': 'May 17 2025 16:33:48 (GMT)', 'scan_started_t': 1747492428, 'scan_finished': 'May 17 2025 16:33:58 (GMT)', 'scan_finished_t': 1747492438, 'found': 4, 'scanned_domain': 'swizosoft.com', 'scan_type': 'light'}, 'data': [{'subdomain': 'ftp.swizosoft.com', 'type': 'IP', 'dest': '89.117.157.252'}, {'subdomain': 'www.swizosoft.com', 'type': 'NAME', 'dest': 'www.swizosoft.com.cdn.hstgr.net'}, {'subdomain': 'www.swizosoft.com.cdn.hstgr.net', 'type': 'IP', 'dest': '92.113.23.61'}, {'subdomain': 'auth.swizosoft.com', 'type': 'IP', 'dest': '89.117.157.252'}], 'seconds': '9.603'}
['ftp.swizosoft.com', 'www.swizosoft.com', 'www.swizosoft.com.cdn.hstgr.net', 'auth.swizosoft.com']
